In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression

In [50]:
teams_half = pd.read_csv('/Users/davidpeterson/Downloads/PIETHON/Baseball_Data/core/TeamsHalf.csv')
batting_p = pd.read_csv('/Users/davidpeterson/Downloads/PIETHON/Baseball_Data/core/BattingPost.csv')
fielding_p = pd.read_csv('/Users/davidpeterson/Downloads/PIETHON/Baseball_Data/core/FieldingPost.csv')
pitching_p = pd.read_csv('/Users/davidpeterson/Downloads/PIETHON/Baseball_Data/core/PitchingPost.csv')
salaries = pd.read_csv('/Users/davidpeterson/Downloads/PIETHON/Baseball_Data/core/Salaries.csv')
pitching_p = pd.read_csv('/Users/davidpeterson/Downloads/PIETHON/Baseball_Data/core/PitchingPost.csv')
teams = pd.read_csv('/Users/davidpeterson/Downloads/PIETHON/Baseball_Data/core/Teams.csv')
series_p = pd.read_csv('/Users/davidpeterson/Downloads/PIETHON/Baseball_Data/core/SeriesPost.csv')

In [53]:
teams.head()

,yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,...,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro
0,1871,NaN,BS1,BNA,NaN,3,31,NaN,20,10,...,24,0.834,Boston Red Stockings,South End Grounds I,NaN,103,98,BOS,BS1,BS1
1,1871,NaN,CH1,CNA,NaN,2,28,NaN,19,9,...,16,0.829,Chicago White Stockings,Union Base-Ball Grounds,NaN,104,102,CHI,CH1,CH1
2,1871,NaN,CL1,CFC,NaN,8,29,NaN,10,19,...,15,0.818,Cleveland Forest Citys,National Association Grounds,NaN,96,100,CLE,CL1,CL1
3,1871,NaN,FW1,KEK,NaN,7,19,NaN,7,12,...,8,0.803,Fort Wayne Kekiongas,Hamilton Field,NaN,101,107,KEK,FW1,FW1
4,1871,NaN,NY2,NNA,NaN,5,33,NaN,16,17,...,14,0.840,New York Mutuals,Union Grounds (Brooklyn),NaN,90,88,NYU,NY2,NY2


In [55]:
teams.columns 
# Drop divID attendecnee / Ghome / franchID / yearID / DivWin  / WCWin / LgWin / WSWin      

Index(['yearID', 'lgID', 'teamID', 'franchID', 'divID', 'Rank', 'G', 'Ghome',
       'W', 'L', 'DivWin', 'WCWin', 'LgWin', 'WSWin', 'R', 'AB', 'H', '2B',
       '3B', 'HR', 'BB', 'SO', 'SB', 'CS', 'HBP', 'SF', 'RA', 'ER', 'ERA',
       'CG', 'SHO', 'SV', 'IPouts', 'HA', 'HRA', 'BBA', 'SOA', 'E', 'DP', 'FP',
       'name', 'park', 'attendance', 'BPF', 'PPF', 'teamIDBR',
       'teamIDlahman45', 'teamIDretro'],
      dtype='object')

In [66]:
teams.isnull().sum()

yearID               0
lgID                50
teamID               0
franchID             0
divID             1517
Rank                 0
G                    0
Ghome              399
W                    0
L                    0
DivWin            1545
WCWin             2181
LgWin               28
WSWin              357
R                    0
AB                   0
H                    0
2B                   0
3B                   0
HR                   0
BB                   1
SO                 120
SB                 126
CS                 832
HBP               1326
SF                1541
RA                   0
ER                   0
ERA                  0
CG                   0
SHO                  0
SV                   0
IPouts               0
HA                   0
HRA                  0
BBA                  0
SOA                  0
E                    0
DP                   0
FP                   0
name                 0
park                34
attendance         279
BPF        

In [84]:
wins = teams['W']
loses = teams['L']
df = teams[['yearID','teamID','Rank','W','L']]

In [102]:
df[df['yearID'] >= 2017].reset_index()

,index,yearID,teamID,Rank,W,L
0,2835,2017,ARI,2,93,69
1,2836,2017,ATL,3,72,90
2,2837,2017,BAL,5,75,87
3,2838,2017,BOS,1,93,69
4,2839,2017,CHA,4,67,95
5,2840,2017,CHN,1,92,70
6,2841,2017,CIN,5,68,94
7,2842,2017,CLE,1,102,60
8,2843,2017,COL,3,87,75
9,2844,2017,DET,5,64,98
